# An Assessment of Health Care Plan Types on ACHP members and Social Risk Factors. By Jake Lieberfarb

The Alliance of Community Health Plans (ACHP) is an advocacy and health care firm located in Washington, D.C. ACHP represents 23 healthcare providers around the US. An analysis was conducted on the types of healthcare plan types available in the US and for ACHP members. Plan types use a system of ratings called "Stars" to evaluate 9 overall measures and 46 different quality measures.Each quality measure is given a rating from 1-5 with 5 being the highest. These ratings are known as "Star ratings". They measure the experiences Medicare beneficiaries have with their healthcare system and health plan. The 46 quality measures are broken down into four groups. For "HD" there are five criteria that rate the overall healthcare experience. There are four "DD" rates the overall drug availability. For example, "DD4" measures 'Drug Safety and Accuracy of Drug Pricing' of a plan. There are 32 "C" rates which measure the clinical aspect of a plan. C03 refers to the availabiliyt of an individual to get the annual flu vaccine. Finally, there are 14 "D" ratings which measures drug availabilities in a plan. All of this information was obtained from the Center for Medicare & Medicaid Services (CMS). [Monthly Enrollment by Plan](https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Plan) contains Medicare enrollment by plan contract and plan type. Next, the [2021 Star Ratings Data Table (Oct 08 2020) (ZIP)](https://www.cms.gov/Medicare/Prescription-Drug-Coverage/PrescriptionDrugCovGenIn/PerformanceData) was included in the analysis as it contains the 9 overall and 46 measure ratings by plan contract. The [Monthly MA Enrollment by State/County/Contract](https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-MA-Enrollment-by-State-County-Contract) was included in the analysis as it contains Medicare enrollment by plan contract and county (FIPS code). The months of October and November were selected to see types of contracts added per month. These files were merged for all plans in the US and ACHP member plans. Finally, social risk factor data (% uninsured, % unemployed, or % Fair or Poor Health) was included on the county level from [Countyhealthrankings.org](https://www.countyhealthrankings.org/explore-health-rankings/rankings-data-documentation) to see if any ACHP members are providing plans for areas with high social risk factors.

In [ ]:
#import necessary modules 
import pandas as pd
from IPython.display import display
import numpy as np
import plotly
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#import all files
#some files could not be read correctly so included to include 'encoding='cp1252'' for file to be properly imported
mr = pd.read_csv("Monthly_Report_By_Plan_2020_10.csv",index_col=0, encoding='cp1252')
ds = pd.read_csv("2021 Star Ratings Data Table - Domain Stars (Oct 08 2020).csv",index_col=0, encoding='cp1252')
ms = pd.read_csv("2021 Star Ratings Data Table - Measure Stars (Oct 08 2020).csv",index_col=0, encoding='cp1252')
ms.index.names = ["Contract Number"]
sr= pd.read_csv("2021 Star Ratings Data Table - Summary Rating (Oct 08 2020).csv",index_col=0, encoding='cp1252')
scc= pd.read_csv("SCC_Enrollment_MA_Alt_2020_10.csv",index_col=2)
scc.index.names = ["Contract Number"]
scc2= pd.read_csv("SCC_Enrollment_MA_Alt_2020_11.csv",index_col=2)
scc2.index.names = ["Contract Number"]
CH = pd.read_excel("2020 County Health Rankings Data - v2.xlsx", sheet_name= 'Ranked Measure Data')
CH.to_csv ("2020 County Health Rankings Data - v2.csv", index = None, header=True)
#using Plotly with login and key
py.sign_in('Jlieberfarb', 'vBRbr33wqGdlxzikJ0Qx')

In [ ]:
#import other files
achp0= pd.read_csv('2021_ACHP_MA_Contract.csv')

In [ ]:
#cleaning the data 
ds.columns = ds.columns.str.replace("'"," ")
ds.rename(columns= {'HD1: Staying Healthy: Screenings, Tests and Vaccines':'HD1',
                  'HD2: Managing Chronic (Long Term) Conditions': 'HD2',
                  'HD3: Member Experience with Health Plan': 'HD3',
                  'HD4: Member Complaints and Changes in the Health Plan s Performance': 'HD4',
                  'HD5: Health Plan Customer Service': 'HD5',
                  'DD1: Drug Plan Customer Service': 'DD1',
                  'DD2: Member Complaints and Changes in the Drug Plan’s Performance': 'DD2',
                  'DD3: Member Experience with the Drug Plan': 'DD3',
                  'DD4: Drug Safety and Accuracy of Drug Pricing': 'DD4'}, inplace= True)

In [ ]:
#clean ACHP data 
achp0.rename(columns={'Contract_ID' :'Contract Number'}, inplace= True)
achp1= achp0.drop(['Organization_Name','Organization_Marketing_Name','Parent_Organization','ACHP_Flag'], axis=1)

In [ ]:
#only want ACHP plans for 2020
achp= achp1[achp1['Year']==2020]

In [ ]:
#cleanning Measures stars data
ms.rename(columns= {"C01: Breast Cancer Screening": "C01",
                    "C02: Colorectal Cancer Screening": "C02",
                    "C03: Annual Flu Vaccine": "C03", 
                    "C04: Improving or Maintaining Physical Health": "C04",
                    "C05: Improving or Maintaining Mental Health": "C05",
                    "C06: Monitoring Physical Activity": "C06",
                    "C07: Adult BMI Assessment": "C07", 
                    "C08: Special Needs Plan (SNP) Care Management": "C08",
                    "C09: Care for Older Adults – Medication Review": "C09",
                    "C10: Care for Older Adults – Functional Status Assessment": "C10", 
                    "C11: Care for Older Adults – Pain Assessment": "C11",
                    "C12: Osteoporosis Management in Women who had a Fracture": "C12",
                    "C13: Diabetes Care – Eye Exam": "C13", 
                    "C14: Diabetes Care – Kidney Disease Monitoring": "C14",
                    "C15: Diabetes Care – Blood Sugar Controlled":  "C15",                   
                    "C16: Rheumatoid Arthritis Management": "C16", 
                    "C17: Reducing the Risk of Falling": "C17",
                    "C18: Improving Bladder Control": "C18",
                    "C19: Medication Reconciliation Post-Discharge": "C19", 
                    "C20: Statin Therapy for Patients with Cardiovascular Disease": "C20",
                    "C21: Getting Needed Care": "C21",                   
                    "C22: Getting Appointments and Care Quickly": "C22", 
                    "C23: Customer Service": "C23",
                    "C24: Rating of Health Care Quality": "C24",
                    "C25: Rating of Health Plan": "C25", 
                    "C26: Care Coordination": "C26",
                    "C27: Complaints about the Health Plan": "C27",                   
                    "C28: Members Choosing to Leave the Plan": "C28", 
                    "C29: Health Plan Quality Improvement": "C29",
                    "C30: Plan Makes Timely Decisions about Appeals": "C30",                   
                    "C31: Reviewing Appeals Decisions": "C31", 
                    "C32: Call Center – Foreign Language Interpreter and TTY Availability": "C32",
                    "D01: Call Center – Foreign Language Interpreter and TTY Availability": "D01",                   
                    "D02: Appeals Auto–Forward": "D02", 
                    "D03: Appeals Upheld": "D03",
                    "D04: Complaints about the Drug Plan": "D04",                   
                    "D05: Members Choosing to Leave the Plan": "D05", 
                    "D06: Drug Plan Quality Improvement":"D06",
                    "D07: Rating of Drug Plan": "D07",                   
                    "D08: Getting Needed Prescription Drugs": "D08",                   
                    "D09: MPF Price Accuracy": "D09", 
                    "D10: Medication Adherence for Diabetes Medications": "D10",
                    "D11: Medication Adherence for Hypertension (RAS antagonists)": "D11",                                                                                                                                                      
                    "D12: Medication Adherence for Cholesterol (Statins)": "D12",                   
                    "D13: MTM Program Completion Rate for CMR": "D13", 
                    "D14: Statin Use in Persons with Diabetes (SUPD)": "D14",
                   }, inplace = True)

In [ ]:
#clean county Health Data and drop unnecissary columns as they would not be useful for graphical analysis
ch=CH.drop(['95% CI - Low', '95% CI - High', 'YPLL Rate (AIAN) 95% CI - Low',
                          'YPLL Rate (AIAN) 95% CI - High','YPLL Rate (Asian) 95% CI - Low',
                          'YPLL Rate (Asian) 95% CI - High', 'YPLL Rate (Black) 95% CI - Low',
                          'YPLL Rate (Black) 95% CI - High', 'YPLL Rate (Hispanic) 95% CI - Low',
                          'YPLL Rate (Hispanic) 95% CI - High','95% CI - Low.1',
                          '95% CI - High.1','Quartile.1','95% CI - Low.2', 'Quartile.2',
                          '95% CI - Low.3', '95% CI - High.3','Quartile.3','Unreliable',
                          '95% CI - Low.4','95% CI - High.4', 'Quartile.4', '% LBW (AIAN) 95% CI - Low',
                          '% LBW (AIAN) 95% CI - High', '% LBW (Asian) 95% CI - Low', 
                           '% LBW (Asian) 95% CI - High', '% LBW (Black) 95% CI - Low',
                          '% LBW (Black) 95% CI - High', 'YPLL Rate (White) 95% CI - Low',
                          'YPLL Rate (White) 95% CI - High', '95% CI - High.2',
                           '% LBW (Hispanic) 95% CI - Low', '% LBW (Hispanic) 95% CI - High',
                           '% LBW (White) 95% CI - Low', '% LBW (White) 95% CI - High',
                           '95% CI - Low.5', '95% CI - High.5', 'Quartile.5', '95% CI - Low.6',
                           '95% CI - High.6', 'Quartile.6', 'Quartile.7', '95% CI - Low.7',
                           '95% CI - High.7', 'Quartile.8','Quartile.9', '95% CI - Low.8',
                           '95% CI - High.8', 'Quartile.10', '95% CI - Low.9', '95% CI - High.9',
                           'Quartile.11', 'Quartile.12', '95% CI - Low.10', '95% CI - High.10',
                           'Quartile.13', 'Teen Birth Rate (AIAN) 95% CI - Low', 
                           'Teen Birth Rate (AIAN) 95% CI - High','Teen Birth Rate (Asian) 95% CI - Low',
                           'Teen Birth Rate (Asian) 95% CI - High', 'Teen Birth Rate (Black) 95% CI - Low',
                           'Teen Birth Rate (Black) 95% CI - High', 'Teen Birth Rate (Hispanic) 95% CI - Low',
                           'Teen Birth Rate (Hispanic) 95% CI - High', 'Teen Birth Rate (White) 95% CI - Low',
                           'Teen Birth Rate (White) 95% CI - High', '95% CI - Low.11', '95% CI - High.11',
                           'Quartile.14', 'Quartile.15', 'Quartile.16', 'Quartile.17', 'Quartile.18', 
                           'Quartile.19', 'Quartile.20', 'Quartile.21', '95% CI - Low.12', '95% CI - High.12',
                           'Quartile.22', 'Quartile.23', '95% CI - Low.13', '95% CI - High.13', 'Quartile.24',
                           'Quartile.25', '95% CI - Low.14', '95% CI - High.14', 'Quartile.26', 'Quartile.27',
                           'Quartile.28', '95% CI - Low.15', '95% CI - High.15',
                           'Quartile.29', 'Injury Death Rate (AIAN) 95% CI - Low',
                           'Injury Death Rate (AIAN) 95% CI - High',
                           'Injury Death Rate (Asian) 95% CI - Low',
                           'Injury Death Rate (Asian) 95% CI - High',
                           'Injury Death Rate (Black) 95% CI - Low',
                           'Injury Death Rate (Black) 95% CI - High', 'Quartile',
                           'Injury Death Rate (Hispanic) 95% CI - Low', '95% CI - High.16',
                           'Injury Death Rate (Hispanic) 95% CI - High', '95% CI - Low.16',
                           'Injury Death Rate (White) 95% CI - Low', 'Quartile.31',
                           'Injury Death Rate (White) 95% CI - High', 'Quartile.30',
                           'Severe Housing Cost Burden 95% CI - Low', 'Severe Housing Cost Burden 95% CI - High',
                           'Overcrowding 95% CI - Low', 'Overcrowding 95% CI - High',
                           'Inadequate Facilities 95% CI - Low', 'Inadequate Facilities 95% CI - High',
                           'Quartile.32', '95% CI - Low.17', '95% CI - High.17', 'Quartile.33', 
                           '% Drive Alone (AIAN) 95% CI - Low', '% Drive Alone (AIAN) 95% CI - High',
                           '% Drive Alone (Asian) 95% CI - Low', '% Drive Alone (Asian) 95% CI - High',
                           '% Drive Alone (Black) 95% CI - Low', '% Drive Alone (Black) 95% CI - High',
                           '% Drive Alone (Hispanic) 95% CI - Low', '% Drive Alone (Hispanic) 95% CI - High',
                           '% Drive Alone (White) 95% CI - Low', '% Drive Alone (White) 95% CI - High',
                           '95% CI - Low.18','95% CI - High.18','Quartile.34',
                           'YPLL Rate (AIAN)','YPLL Rate (Asian)','YPLL Rate (Black)','YPLL Rate (Hispanic)',
                           'YPLL Rate (White)','% Drive Alone (AIAN)','% Drive Alone (Asian)',
                           '% Drive Alone (Black)', '% Drive Alone (Hispanic)', '% Drive Alone (White)',
                           '% LBW (AIAN)', '% LBW (Asian)', '% LBW (Black)', '% LBW (Hispanic)',
                           'Teen Birth Rate (AIAN)', 'Teen Birth Rate (Asian)', 'Teen Birth Rate (Black)',
                           'Teen Birth Rate (Hispanic)', 'Teen Birth Rate (White)',
                           'Preventable Hosp. Rate (AIAN)', 'Preventable Hosp. Rate (Asian)',
                           'Preventable Hosp. Rate (Black)', 'Preventable Hosp. Rate (Hispanic)',
                           'Preventable Hosp. Rate (White)', '% Screened (AIAN)', '% Screened (Asian)',
                           '% Screened (Black)', '% Screened (Hispanic)', '% Screened (White)',
                           '% Vaccinated (AIAN)', '% Vaccinated (Asian)', '% Vaccinated (Black)',
                           '% Vaccinated (Hispanic)', '% Vaccinated (White)', '% Children in Poverty (AIAN)',
                           '% Children in Poverty (Asian)', '% Children in Poverty (Black)',
                           '% Children in Poverty (Hispanic)', '% Children in Poverty (White)',
                           '80th Percentile Income', '20th Percentile Income','Injury Death Rate (AIAN)',
                           'Injury Death Rate (Asian)', 'Injury Death Rate (Black)',  
                           'Injury Death Rate (Hispanic)', 'Injury Death Rate (White)','Average Daily PM2.5',
                           '% LBW (White)'], axis=1)
pd.set_option('display.max_columns', 200)

In [ ]:
#reset indecies so can merge dataframes later
mr0= mr.reset_index()
ms0= ms.reset_index()
ds0= ds.reset_index()
sr0= sr.reset_index()
scc= scc.reset_index()
scc2= scc2.reset_index()

In [ ]:
#realized dataframes were not merging bc there was a blank space in Contract Number for dataframes
sr0['Contract Number'].iloc[0]

In [ ]:
#removes all leading and trailing white spaces form 'Contract Number'
ds0['Contract Number']= ds0['Contract Number'].str.strip()
ms0['Contract Number']= ms0['Contract Number'].str.strip()
sr0['Contract Number']= sr0['Contract Number'].str.strip()

In [ ]:
#merge domain starts with monthly report 
mr1= mr0.merge(ds0[['Contract Number', 'HD1','HD2', 'HD3', 'HD4', 'HD5', 'DD1', 'DD2', 'DD3', 'DD4']], 
              on= 'Contract Number')

In [ ]:
#add Measure Stars data to monthly report
mr2= mr1.merge(ms0[['Contract Number','C01','C02','C03','C04','C05','C06','C07','C08','C09','C10','C11','C12','C13','C14','C15',
                   'C16','C17','C18','C19','C20','C21','C22','C23','C24','C25','C26','C27','C28','C29','C30',
                   'C31','C32','D01','D02','D03','D04','D05','D06','D07','D08','D09','D10','D11','D12','D13',
                   'D14']], on= 'Contract Number')
pd.set_option('display.max_columns', 100)

In [ ]:
#add summary rating to monthly report
mr3=mr2.merge(sr0[['Contract Number','SNP','2018 Disaster %','2019 Disaster %', '2021 Part C Summary', 
                  '2021 Part D Summary', '2021 Overall']], on= 'Contract Number')
pd.set_option('display.max_columns', 100)

In [ ]:
#drop repetative columns for merge later on
mr4=mr3.drop(['Organization Name', 'Organization Type'], axis=1)

In [ ]:
mr_cal= mr4.drop(['Offers Part D',
                  'Organization Marketing Name','Parent Organization',
                  'Contract Effective Date','SNP','2018 Disaster %',
                  '2019 Disaster %','2021 Part C Summary', '2021 Part D Summary'], axis=1)

#merge for 2020 October (scc) and 2020 November (scc2)
##October
mr_cal1= mr_cal.merge(scc[['Contract Number', 'County', 'State', 'Enrolled']], how='left', on= 'Contract Number')
#drop duplicate contract numbers 
mr_cal1= mr_cal1.drop_duplicates("Contract Number", keep='first')
##November
mr_cal2= mr_cal.merge(scc2[['Contract Number', 'County', 'State', 'Enrolled']], how='left', on= 'Contract Number')
#drop duplicate contract numbers 
mr_cal2= mr_cal2.drop_duplicates("Contract Number", keep='first')

# Overview of types of plans and programs offered in America

Pie and bar charts were constructed to provide an overview of the types of contracts offered in America. Two Pie charts with one for October and one for November were constructed for the number of enrollees by each plan type. Next, a bar chart for the count of contract enrollments was made to see the types of plans offered in each state. The average star ratings and percentage of 4 and 5 ratings were calculated by plan type for the overall and quality measures. This was done to provide a hollistic understanding of what the focus is clinical and drug wise for each plan type offered. 

In [ ]:
#preping pie chart data
#remove plan type which have no enrollment data
#mr_cal1=mr_cal1[mr_cal1['Enrollment']!='*']
mr_cal1['Enrolled']=pd.to_numeric(mr_cal1['Enrolled'])
pie_data = mr_cal1.groupby(["Plan Type"], as_index=False).agg({'Enrolled': 'sum'})

In [ ]:
pie_data.head(10).sort_values(by='Enrolled', ascending=False)

In [ ]:
#NOTE
#pie chart for plan types October 
fig = go.Figure()

fig.add_trace(
            go.Pie(labels=pie_data['Plan Type'], 
                    values= pie_data['Enrolled'], 
                    hoverinfo='label+percent+value',
                    textinfo= 'percent'))
#puts the text inside each slice of the pie 
fig.update_traces(textposition='inside')
#minsize gives you the minumum size that a percentage would appear in the pie 
#if percentage is too small it won't be visible, you have to hover over it
fig.update_layout(uniformtext_minsize=10, title = 'October 2020 New Contracts by Plan Types', height = 600)
fig.show()

In [ ]:
mr_cal2['Enrolled']=pd.to_numeric(mr_cal1['Enrolled'])
pie_data2 = mr_cal2.groupby(["Plan Type"], as_index=False).agg({'Enrolled': 'sum'})
pie_data2.head(10).sort_values(by='Enrolled', ascending=False)

In [ ]:
#pie chart for plan types November
fig = go.Figure()

fig.add_trace(
            go.Pie(labels=mr_cal2['Plan Type'], 
                    values= mr_cal2['Enrolled'], 
                    hoverinfo='label+percent+value',
                    textinfo= 'percent'))

fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=10, title = 'November 2020 New Contracts by Plan Types', height = 600)
fig.show()

In [ ]:
# create empty list
integer_check = []
# columns that are integers in dataframe
Col_name = ['HD1', 'HD2', 'HD3', 'HD4', 'HD5',
       'DD1', 'DD2', 'DD3', 'DD4', 'C01', 'C02', 'C03', 'C04', 'C05', 'C06',
       'C07', 'C08', 'C09', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16',
       'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26',
       'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'D01', 'D02', 'D03', 'D04',
       'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', '2021 Overall']

In [ ]:
#make sure all columns are integers
#coerce is included to set any invalid parsings to NAN in the dataframe
for col in Col_name:
    mr_cal[col] = pd.to_numeric(mr_cal[col],errors="coerce")

In [ ]:
#only want the average of domains
avg=mr_cal.drop(['Plan ID'], axis=1)[:-1]
#need to create custom average so function will avoid null values
def custom_mean(df): 
    return df.mean(skipna=True)
avg=avg.groupby(["Plan Type"], as_index=False).agg(custom_mean)
#make overall avererge of each plan type 
avg['row average']=avg.mean(axis=1)

In [ ]:
bar_data= avg.copy()
#only want to include HD,C, and D data to be analyzed
columns = list(bar_data.columns)[1:]

fig = go.Figure()

for col in columns:
    fig.add_trace(
                go.Bar(x=bar_data.sort_values(
                        by= col, ascending = False)['Plan Type'],
                       y= bar_data.sort_values(
                        by= col, ascending = False)[col], 
                       visible= False,
                      marker_color= 'blue'))

buttons= []

for i in range(len(columns)):
    visible = [False]*len(columns)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False
#creating the dropdown menu     
    buttons.append(dict(label = columns[i],
                        method = 'update',           
                        args = [{'visible': visible}]))
#updating the menu so it has the buttoms      
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = 'Average Rating of Plan Types by Quality Measures ',
                 height = 540)
fig.show()

In [ ]:
#find percentage of each plan that has 4 5 stars for each plan
datf=pd.DataFrame()
for name in Col_name:
    #creates a values counts dataframe based on the given column from HD1 through
    df=mr_cal[name].value_counts().reset_index()
    #adds all 
    datf=pd.concat([datf,df], ignore_index=True, axis=0)

In [ ]:
#counting number of quality measures by plan type
def custom_count(df): 
    return df.count()
mr_cal.groupby(["Plan Type"]).agg(custom_count)

In [ ]:
#creates dataframe and goes through each column by plan types and counts the number of each star
masterdf=pd.DataFrame()
#looping through all unique plan types
for plan in mr_cal['Plan Type'].unique():
    #creating datafrome with values only for a specific plan type
    df=mr_cal[mr_cal['Plan Type']== plan]
    #loop through all the columns
    for name in Col_name:
        #for each column in the dataframe, it goes through and creates a value counts
        df1=df[name].value_counts().reset_index()
        #creating a new column in the dataframe that has the plan type
        df1['Plan Type']= plan
        # creating a new column in the dataframe that has the original column name (HD1, HD2) as its value
        df1['col']= name
        #changes the column names to what the columns are suppose to be
        df1.columns=['Stars','Count','Plan Type','Column_Name']
        #stacks all relevant data
        masterdf=pd.concat([masterdf, df1])
#removes any NAN values
masterdf=masterdf.fillna(0)

In [ ]:
#goes through all plan types and their quality measures and counts numbers of 4s and 5s
percents=[]
#only want unique plan types
for plan in masterdf['Plan Type'].unique():
    #selects data from masterdf by plan type
    df=masterdf[masterdf['Plan Type']== plan]
    #loops through all the unique column names
    for name in df['Column_Name'].unique():
        #matches one column name at a time with the name it is looping through
        df2=df[df['Column_Name']== name]
        #selecting the total number of stars
        total= df2["Count"].sum()
        #counts the number of 4s and 5s
        and4_5= df2[df2['Stars']>= 4]['Count'].sum()
        #calculates the percent of 4s and 5s
        percent= and4_5/total*100 
        #creates a tuple of all relevant info
        percents.append((plan, name, percent, and4_5))
#puts all data in a dataframe
perc_df=pd.DataFrame(percents,columns=['Plan Type', 'Column_Name', 'Percentage', 'count'])

In [ ]:
df2.head()

In [ ]:
bar_data= perc_df.copy()

columns = list(perc_df['Plan Type'].unique())

fig = go.Figure()

for col in columns:
    fig.add_trace(
                go.Bar(x=bar_data[bar_data['Plan Type']==col].sort_values(
                        by= 'Percentage', ascending = False)['Column_Name'],
                       y= bar_data[bar_data['Plan Type']==col].sort_values(
                        by= 'Percentage', ascending = False)['Percentage'], 
                       visible= False,
                      marker_color= 'purple'))

buttons= []

for i in range(len(columns)):
    visible = [False]*len(columns)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False
#creating the dropdown menu     
    buttons.append(dict(label = columns[i],
                        method = 'update',           
                        args = [{'visible': visible}]))
#updating the menu so it has the buttoms      
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = 'Percentage of 4 and 5 Star Ratings per Plan',
                 height = 540)
fig.show()

In [ ]:
#drop NA values 
mr_cal1= mr_cal1.dropna(axis=0, subset=['State'])

There were significanlty more contracts added in October than in November. However, in both cases, HMO was the most common type. When looking at the highest average rating of plan types by Quality Measure, HMOs and 1876 cost appear to have high ratings for each quality measure. However, there is still a wide variety of plan types that offer different levels are ratings. When looking at the percent of fours and fives star ratings by plan type,  HMOs, Local PPOs, and Regional PPOs offer the wides range of coverage and overall have a majority of stars that are fours and fives. 1876 Cost has the most quality scores that are only fours and fives but still does not have as much variety as HMOs and local and Regional PPOs.  HMO/HMOPOS has the least number of quality measure offered as part of its plan type and least number of fours and fives ratings. 

# Overview of types of ACHP plans and programs offered 

Next, an overview of the types of health care contracts offered by ACHP members was constructed to understand which plan types are most common amongst ACHP members and if these plan types are focusing on specific quality measures. Pie and bar charts were constructed to analyze the data. A pie chart was constructed to show the count of plan types offered by ACHP members. Another pie chart was included to show the breakdown of which plan types each member offers. bar charts were included to show the total count of contracts offered by ACHP members, the average of quality scores by member and percent of 4s and 5s for quality scores by member.

In [ ]:
ACHP= mr_cal2.merge(achp, on= 'Contract Number')
#need to remove any duplicate contract numbers
ACHP.drop_duplicates("Contract Number", keep='first', inplace=True)

In [ ]:
mr_cal2

In [ ]:
pie_data3 = ACHP.groupby(["Plan Type"], as_index=False).agg({'Plan ID': 'count'})

In [ ]:
pie_data3.head(10).sort_values(by='Plan ID', ascending=False)

In [ ]:
#pie chart for ACHP plan types  
fig = go.Figure()

fig.add_trace(go.Pie(labels=pie_data3['Plan Type'], 
                       values= pie_data3['Plan ID'], 
                       hoverinfo='label+percent+value',
                       textinfo= 'percent'))

fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=10, title = 'Types of Contracts Offered by ACHP Members', height = 600)
fig.show()

In [ ]:
ACHP.head()

In [ ]:
#FIX pie chart to show breakdown of Plan Types by rollout 
#pie chart types of plans offered by ACHP members
rollups = list(ACHP['Rollup'].unique()) 

ACHP_bar= ACHP.groupby(['Rollup', 'Plan Type'], as_index=False).agg({'Plan ID': 'count'})

fig = go.Figure()

for rollup in rollups:
    fig.add_trace(
                go.Pie(labels=ACHP_bar[ACHP_bar['Rollup'] == rollup]['Plan Type'], 
                       values= ACHP_bar[ACHP_bar['Rollup'] == rollup]['Plan ID'], 
                       hoverinfo='label+percent+value',
                       textinfo= 'none',
                       visible= False))

    
buttons= []

for i in range(len(rollups)):
    visible = [False]*len(rollups)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False

    buttons.append(dict(label = rollups[i],
                        method = 'update',           
                        args = [{'visible': visible}]))
  
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = 'Plan Types Offered by ACHP Members',
                 height = 600)
fig.show()

In [ ]:
#Which ACHP member has the most contracts?
ACHP['Enrollment']=pd.to_numeric(ACHP['Enrolled'])
bar_chart4 = ACHP.groupby(["Rollup"], as_index=False).agg({'Plan ID': 'count'})

In [ ]:
fig = go.Figure()
fig.add_trace( go.Bar(x= bar_chart4.sort_values(by ='Plan ID', ascending=False)['Rollup'], 
    y= bar_chart4.sort_values(by ='Plan ID', ascending=False)['Plan ID'], marker_color='Orange'))

fig.update_layout(title = 'Count of Total Plan Types Offered by ACHP Member Plans', height = 540)
fig.update_xaxes(title_text = "ACHP Members")
fig.update_yaxes(title_text = "Count of Plan Types")

In [ ]:
# create empty list
integer_check = []
# columns that are integers in dataframe
Col_name = ['HD1', 'HD2', 'HD3', 'HD4', 'HD5',
       'DD1', 'DD2', 'DD3', 'DD4', 'C01', 'C02', 'C03', 'C04', 'C05', 'C06',
       'C07', 'C08', 'C09', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16',
       'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26',
       'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'D01', 'D02', 'D03', 'D04',
       'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', '2021 Overall']
for col in Col_name:
    ACHP[col] = pd.to_numeric(ACHP[col],errors="coerce")

In [ ]:
avg2=ACHP.drop(['Plan ID','Plan Name','Enrollment','County','State','Enrolled','Year'], axis=1)

In [ ]:
def custom_mean(df): 
    return df.mean(skipna=True)
avg2=avg2.groupby(["Rollup"], as_index=False).agg(custom_mean)
avg2['row average']=avg2.mean(axis=1)

In [ ]:
avg2.sort_values(by='row average', ascending=False).head(23)

In [ ]:
#create bar chart of ACHP averages
ACHP_bar2= avg2.copy()
#only want to include HD,C, and D data to be analyzed
columns = list(ACHP_bar2.columns)[1:]

fig = go.Figure()

for col in columns:
    fig.add_trace(
                go.Bar(x=ACHP_bar2.sort_values(
                        by= col, ascending = False)['Rollup'],
                       y= ACHP_bar2.sort_values(
                        by= col, ascending = False)[col], 
                       visible= False,
                      marker_color= 'orange'))

buttons= []

for i in range(len(columns)):
    visible = [False]*len(columns)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False
    
    buttons.append(dict(label = columns[i],
                        method = 'update',           
                        args = [{'visible': visible}]))
      
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = 'Average Rating of ACHP Plan Types by Domain ',
                 height = 540)
fig.show()

In [ ]:
#Overall averages by plan types
ACHP_bar2[['Rollup', 'row average']].sort_values(by= 'row average', ascending = False)

In [ ]:
datf=pd.DataFrame()
for name in Col_name:
    df=mr_cal[name].value_counts().reset_index()
    datf=pd.concat([datf,df], ignore_index=True, axis=0)

In [ ]:
def custom_count(df): 
    return df.count()
ACHP.groupby(["Rollup"]).agg(custom_count)

In [ ]:
masterdf=pd.DataFrame()
for plan in ACHP['Rollup'].unique():
    df=ACHP[ACHP['Rollup']== plan]
    for name in Col_name:
        df1=df[name].value_counts().reset_index()
        df1['Rollup']= plan
        df1['col']= name
        df1.columns=['Stars','Count','Rollup','Column_Name']
        masterdf=pd.concat([masterdf, df1])
masterdf=masterdf.fillna(0)

In [ ]:
percents=[]
for plan in masterdf['Rollup'].unique():
    df=masterdf[masterdf['Rollup']== plan]
    for name in df['Column_Name'].unique():
        df2=df[df['Column_Name']== name]
        total= df2["Count"].sum()
        and4_5= df2[df2['Stars']>= 4]['Count'].sum()
        percent= and4_5/total*100 
        percents.append((plan, name, percent, and4_5))
perc_df2=pd.DataFrame(percents,columns=['Rollup', 'Column_Name', 'Percentage', 'count'])

In [ ]:
bar_data= perc_df2.copy()

columns = list(perc_df2['Rollup'].unique())

fig = go.Figure()

for col in columns:
    fig.add_trace(
                go.Bar(x=bar_data[bar_data['Rollup']==col].sort_values(
                        by= 'Percentage', ascending = False)['Column_Name'],
                       y= bar_data[bar_data['Rollup']==col].sort_values(
                        by= 'Percentage', ascending = False)['Percentage'], 
                       visible= False,
                      marker_color= 'orange'))

buttons= []

for i in range(len(columns)):
    visible = [False]*len(columns)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False
#creating the dropdown menu     
    buttons.append(dict(label = columns[i],
                        method = 'update',           
                        args = [{'visible': visible}]))
#updating the menu so it has the buttoms      
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = 'Percentage of 4 and 5 Star Ratings per Plan (ACHP)',
                 height = 540)
fig.show()

The most common plan type offered by ACHP members are HMOs at 36 types followed by 21 PPOS. some plans such as kaiser only offers HMOs. Healthpartners has Local PPOs, 1876 Cost, HMO, and Medicare Prescription Drug Plan. A bar chart of the count of plan types by ACHP member revealed that Healthalliance and Kaiser had the most at seven which was followed by UPMC at five. Overall most ACHP plans offer 2-3 types of contracts. Upon investigating the average of quaity measurements it appears that the ACHP members with the highest 2021 OVerall stars rating are Kaiser at 4.928, CDPHP at 4.75, Martins Point at 4.75, and Health Partners at 4.625. Kaiser and CDPHP has 43 quality measures that were either 4s or 5s. Martins point had 44 quality measure that were either 4s or 5s. CommunityCare and Presbyterian have the lowest 2021 Overall stars rating at 3.5465 and 3.3884, respectivley. CommunityCare and Presbyterian have  21 quality measures of only 4s and 5s.  

# State and County Social Risks 

Next, the social risk scores was explored on the state and county level. The three main risk scores of interest are % Fair or Poor Health, % Uninsured, and % Unemployed. These three risk scores were given the main focus in this analysis as they would be key criteria used in deciding if an ACHP member would want to take on the risk of providing insurance plans in this area. A choropleth map was included of social risk factors by state. A bar chart shows the counties and states with the highest social risk factors. Choropleth maps for the three risk factors was also constructed on the county level. A weighted stars value was constructed by taking the total number of enrollees available in each county and multiplying it by the overall star rating for each county and then summing the results for each county. A choropleth map for weighted stars by county was constructed to see if more urban or rural areas have different weighted stars. These charts are useful in understanding the spread of certain social risk factors such as finding the state with the most unemployed or uninsured. 

In [ ]:
#add state code to dataframe ch for choropleth maps 
state_codes = {
    'Pennsylvania': 'PA', 'Florida': 'FL', 'Alaska': 'AK', 'Kentucky': 'KY', 
    'Hawaii': 'HI', 'Nebraska': 'NE', 'Missouri': 'MO', 'Ohio': 'OH', 
    'Alabama': 'AL', 'Rhode Island': 'RI', 'South Dakota': 'SD', 
    'Colorado': 'CO', 'New Jersey': 'NJ', 'Washington': 'WA', 
    'North Carolina': 'NC', 'New York': 'NY', 'Texas': 'TX',   
    'District of Columbia' : 'DC','Mississippi': 'MS', 'Oklahoma': 'OK', 
    'Delaware': 'DE', 'Minnesota': 'MN', 'Illinois': 'IL', 'Arkansas': 'AR', 
    'New Mexico': 'NM', 'Indiana': 'IN', 'Maryland': 'MD', 'Louisiana': 'LA', 
    'Idaho': 'ID', 'Wyoming': 'WY', 'Tennessee': 'TN', 'Arizona': 'AZ', 
    'Iowa': 'IA', 'Michigan': 'MI', 'Kansas': 'KS', 'Utah': 'UT', 
    'Virginia': 'VA', 'Oregon': 'OR', 'Connecticut': 'CT', 'Montana': 'MT', 
    'California': 'CA', 'Massachusetts': 'MA', 'West Virginia': 'WV', 
    'South Carolina': 'SC', 'New Hampshire': 'NH', 'Wisconsin': 'WI',
    'Vermont': 'VT', 'Georgia': 'GA', 'North Dakota': 'ND', 
    'Nevada': 'NV', 'Maine': 'ME'}

statekeys= state_codes.keys()
code_values=state_codes.values()
state_codes=dict(zip(statekeys,code_values))
ch['STATE_CODE'] = ch['State'].map(state_codes)

In [ ]:
#prep data
#only have data on state, not the county
ch_country= ch[ch['County'].isnull()]
#remove overall state info
ch_clean=ch[~ch['County'].isnull()]
#create column that has both county and state
ch_clean['County&State']=ch_clean['County']+ ', ' + ch_clean['State']

In [ ]:
columns= list(ch_country.columns)[3:-1]

chor_data= ch_country.copy()

fig = go.Figure()

for col in columns:
    fig.add_trace(
            go.Choropleth(
                locations=ch_country['STATE_CODE'],
                z = ch_country[col], 
                locationmode = 'USA-states',
                text=ch_country['State'],
                colorscale = 'Reds',
                hoverinfo='location+z',
                colorbar_title = '{} by state'.format(col),
                visible= False))

buttons= []

for i in range(len(columns)):
    visible = [False]*len(columns)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False
#creating the dropdown menu     
    buttons.append(dict(label = columns[i],
                        method = 'update',           
                        args = [{'visible': visible}]))
#updating the menu so it has the buttoms      
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = 'Social Risk Factors by State',
                 height = 540,
                 geo_scope = 'usa')
fig.show()

In [ ]:
#bar chart of personal health 
bar_data= ch_clean.copy()

#Only want column names after 'County' in drop down
columns = list(bar_data.columns)[4:] 

fig = go.Figure()

for col in columns:
    fig.add_trace(
                go.Bar(x=bar_data.sort_values(
                        by= col, ascending = False)['County&State'].head(10),
                       y= bar_data.sort_values(
                        by= col, ascending = False)[col].head(10), 
                       visible= False,
                       marker_color= 'red'))

buttons= []

for i in range(len(columns)):
    visible = [False]*len(columns)
    if i == 0: 
        visible[i]= True
    else:
        visible[i] = True
        visible[i-1] = False
#creating the dropdown menu     
    buttons.append(dict(label = columns[i],
                        method = 'update',           
                        args = [{'visible': visible}]))
#updating the menu so it has the buttoms      
fig.update_layout(updatemenus=list([
    dict(buttons = buttons)]))

fig.update_layout(title = 'Top 10 Counties by Personal Health Statistics',
                 height = 540)
fig.show()

In [ ]:
scc["Plan Type"].value_counts()

In [ ]:
#need to reset index for merge below
scc=scc.reset_index()

In [ ]:
dv= scc.merge(mr4[['Contract Number','2021 Overall']],how= 'left', on= 'Contract Number')
#only want ints in column '2021 Overall'
dv1= dv[(dv['2021 Overall']!='Plan too new to be measured') & (dv['2021 Overall']!= 'Not enough data available')]
dv1=dv1.drop_duplicates(subset=['Contract Number', 'County'], keep='first')

In [ ]:
total_by_county= dv1.groupby('FIPS Code').sum().reset_index()
dv1= dv1.merge(total_by_county[['FIPS Code', 'Enrolled']], 
               on= 'FIPS Code', suffixes= ('_plan','_total'))

In [ ]:
dv1['weights']= dv1['Enrolled_plan']/dv1['Enrolled_total']
dv1['Weighted Stars']= dv1['weights']*pd.to_numeric(dv1['2021 Overall'])
dv2= dv1.groupby('FIPS Code').agg({'County': 'first', 'State': 'first',
                                   'Enrolled_plan':'sum', 'Weighted Stars' : 'sum'})
ch.rename(columns={'FIPS' :'FIPS Code'}, inplace= True)
dv2.reset_index()
dv3=dv2.merge(ch, on= 'FIPS Code')

In [ ]:
fig = ff.create_choropleth(
    fips=dv3['FIPS Code'], values=dv3['Weighted Stars'],
    title='Weighted Stars by County',
    legend_title='Weighted Stars',binning_endpoints=list(np.arange(0,5.5,.5)))


fig.layout.template= None
fig.show()

In [ ]:
fig = ff.create_choropleth(
    fips=dv3['FIPS Code'], values=dv3['% Fair or Poor Health'],
    title='% Fair or Poor Health by County',
    legend_title='% Fair or Poor Health',binning_endpoints=list(np.arange(0,25,.5)))

fig.layout.template= None
fig.show()

In [ ]:
fig = ff.create_choropleth(
    fips=dv3['FIPS Code'], values=dv3['% Uninsured'],
    title='% Uninsured by County',
    legend_title='% Uninsured',binning_endpoints=list(np.arange(0,35,.5)))

fig.layout.template= None
fig.show()

In [ ]:
fig = ff.create_choropleth(
    fips=dv3['FIPS Code'], values=dv3['% Unemployed'],
    title='% Unemployed by County',
    legend_title='% Unemployed',binning_endpoints=list(np.arange(0,10,.1)))

fig.layout.template= None
fig.show()

For % fair or poor health, the Choropleth map revealed that the states with the highest percentage of this social risk were states found in the Southeast such as Mississippi at 24.148%, Kentucky at 23.8675%, and West Virginia at 23.8994%. The state with the lowest percent of fair or poor health was Minnesota at 12.1324%. The top 10 counties with the highest % of fair or poor health were from Texas, Mississippi, and Alaska. The highest of which was Zavala, Texas at 40.9907%. For % Uninsured, the state choropleth map showed a trend of states in the Southeast to have the highest percentage. Specifically, Texas had 19.3911% and Georgia had 15.5302%. The states with the lowest percentage of uninsured was Hawaii at 4.6321% and Minnesota at 5.085%. When looking at the county level for percent unemployed  was Gaines, Texas at 33.7496%. For percent unemployed, there was no real recognizable pattern. the state with the highest unemployment was Alsaska at 6.5875% an West Virginia at 5.2556%. The state with the lowest unemployement was Hawaii at 2.4468% and Iowa at 2.5236%. The counties with the highest unemployment percentage was Kusilvak, Alaska at 19.9040% The county chropleth map shows that in general unemployment seems to be around no greater than 4%, but some states such as Arizona and Mississippi seem to have some counties with much higher rates of unemployment. The weighted stars choropleth has more urban counties with a higher weighted stars rating than more urban counties. By selecting "4.5-5.0" under the weighted stars legend, you would see that counties around Boston or San Francisco would disappear. If then select "4.0-4.5", you will notice that more rural counties disappear and this trend continues as you move down the weighted stars legend.  Most of the counties have stars ratings between 3.0-5.0. Overall, it appears that Texas has the highest percentage of social risk factors and Hawaii has the lowest.  

# Assessing the Weighted Stars Score of ACHP members on %Fair or Poor Health,  % Uninsured, and % Unemployed by counties 

Finally, an assesment was made comparing the Weighted Stars for ACHP members to the social risk factors of %fair or poor health, %uninsured, and %unemployed. County Choropleths were constructed for ACHP member weighted stars and the three social risk factors. bar charts were built to show which ACHP member has the most contracts to offer highly affected social risk counties. The count of ACHP contracts that serve more than 15% fair or poor health, 15% uninsured, and 5% unemployed was calculated to see if any ACHP members are taking on a greater social risk in the health plans they offer. Then each social risk was compared to the weighted stars on a scatter plot to see if there is any correlation between a weighted stars and increase in a specific social risk.

In [ ]:
#weighted stars for ACHP plans 
#takes into account number of people enrolled on each plan 
DV= scc.merge(ACHP[['Contract Number','2021 Overall']],how= 'left', on= 'Contract Number')
#only want ints in column '2021 Overall'
DV1= DV[(DV['2021 Overall']!='Plan too new to be measured') & (DV['2021 Overall']!= 'Not enough data available')]
DV1=DV1.drop_duplicates(subset=['Contract Number', 'County'], keep='first')

In [ ]:
Total_by_county= DV1.groupby('FIPS Code').sum().reset_index()
DV1= DV1.merge(Total_by_county[['FIPS Code', 'Enrolled']], 
               on= 'FIPS Code', suffixes= ('_plan','_total'))

In [ ]:
#weight created from dividing number of people enrolled by the total number of people enrolled on the plan
DV1['weights']= DV1['Enrolled_plan']/DV1['Enrolled_total']
DV1['Weighted Stars']= DV1['weights']*pd.to_numeric(DV1['2021 Overall'])
DV2= DV1.groupby('FIPS Code').agg({'County': 'first', 'State': 'first',
                                   'Enrolled_plan':'sum', 'Weighted Stars' : 'sum'})
ch.rename(columns={'FIPS' :'FIPS Code'}, inplace= True)
DV2.reset_index()
DV3=DV2.merge(ch, on= 'FIPS Code')
DV3= DV3.drop(['County_x'], axis=1)

In [ ]:
fig = ff.create_choropleth(
    fips=DV3['FIPS Code'], values=DV3['Weighted Stars'],
    title='Weighted Stars by County for ACHP Plans',
    legend_title='Weighted Stars',binning_endpoints=list(np.arange(0.0,5.5,.5)))
fig.layout.template= None
fig.show()

In [ ]:
WS= ACHP[['County','State','Rollup', 'Enrolled']].merge(DV3, how='left',left_on='County', right_on= 'County_y')
WS['>15 Poor Health']= WS[WS['% Fair or Poor Health']>15]['% Fair or Poor Health']
WS= WS[WS['Weighted Stars']!=0]
WS1=WS.copy()

In [ ]:
WS1=WS.groupby("Rollup", as_index=False).agg({'>15 Poor Health' : 'count'})
fig = go.Figure()
fig.add_trace( go.Bar(x= WS1.sort_values(by ='>15 Poor Health', ascending=False)['Rollup'], 
    y= WS1.sort_values(by ='>15 Poor Health', ascending=False)['>15 Poor Health'],marker_color= 'Purple'))
fig.update_layout(title = 'Count of ACHP Contracts with Greater than 15% Fair or Poor Health', height = 540)
fig.update_xaxes(title_text = "ACHP Members")
fig.update_yaxes(title_text = "Count of Contracts")

In [ ]:
fig = go.Figure(data=go.Scatter(x=WS['% Fair or Poor Health'],
                                y=WS['Weighted Stars'],
                                mode='markers',
                                text=WS['Rollup'],
                                marker_color= 'purple'))
fig.update_layout(title = 'ACHP Weighted Stars vs.% Fair or Poor Health', height = 540)
fig.update_xaxes(title_text = "% Fair or Poor Health")
fig.update_yaxes(title_text = "Weigthed Stars")
fig.show()

In [ ]:
# correlation between Weighted Stars and % Fair or Poor Health 
WS['% Fair or Poor Health'].corr(WS['Weighted Stars'])

In [ ]:
WS= ACHP[['County','State','Rollup', 'Enrolled']].merge(DV3, how='left',left_on='County', right_on= 'County_y')
WS['>15 Uninsured']= WS[WS['% Uninsured']>15]['% Uninsured']
WS= WS[WS['Weighted Stars']!=0]
WS2=WS.copy()

In [ ]:
WS2=WS.groupby("Rollup", as_index=False).agg({'>15 Uninsured' : 'count'})
fig = go.Figure()
fig.add_trace( go.Bar(x= WS2.sort_values(by ='>15 Uninsured', ascending=False)['Rollup'], 
    y= WS2.sort_values(by ='>15 Uninsured', ascending=False)['>15 Uninsured'],marker_color= 'purple'))
fig.update_layout(title = 'ACHP Contracts who Serve Greater than 15% Uninsured', height = 540)
fig.update_xaxes(title_text = "ACHP Members")
fig.update_yaxes(title_text = "Count of Contracts")

In [ ]:
fig = go.Figure(data=go.Scatter(x=WS['% Uninsured'],
                                y=WS['Weighted Stars'],
                                mode='markers',
                                text=WS['Rollup'],
                                marker_color= 'purple'))
fig.update_layout(title = 'ACHP Weighted Stars vs.% Uninsured', height = 540)
fig.update_xaxes(title_text = "% Uninsured")
fig.update_yaxes(title_text = "Weigthed Stars")
fig.show()

In [ ]:
# correlation between Weighted Stars and % uninsured
WS['% Uninsured'].corr(WS['Weighted Stars'])

In [ ]:
WS= ACHP[['County','State','Rollup', 'Enrolled']].merge(DV3, how='left',left_on='County', right_on= 'County_y')
WS['>5 Unemployed']= WS[WS['% Unemployed']>5]['% Unemployed']
WS= WS[WS['Weighted Stars']!=0]
WS3=WS.copy()

In [ ]:
WS3=WS.groupby("Rollup", as_index=False).agg({'>5 Unemployed' : 'count'})
fig = go.Figure()
fig.add_trace( go.Bar(x= WS3.sort_values(by ='>5 Unemployed', ascending=False)['Rollup'], 
    y= WS3.sort_values(by ='>5 Unemployed', ascending=False)['>5 Unemployed'], marker_color= 'purple'))
fig.update_layout(title = 'ACHP Contracts that Serve Greater than 5% Unemployed', height = 540)
fig.update_xaxes(title_text = "ACHP Members")
fig.update_yaxes(title_text = "Count of Contracts")

In [ ]:
fig = go.Figure(data=go.Scatter(x=WS['% Unemployed'],
                                y=WS['Weighted Stars'],
                                mode='markers',
                                text=WS['Rollup'],
                                marker_color= 'purple'))
fig.update_layout(title = 'ACHP Weighted Stars vs.% Unemployed', height = 540)
fig.update_xaxes(title_text = "% Unemployed")
fig.update_yaxes(title_text = "Weigthed Stars")
fig.show()

In [ ]:
# correlation between Weighted Stars and % unemployed
WS['% Unemployed'].corr(WS['Weighted Stars'])

# Conclusion

The weighted stars by county choropleth showed that ACHP members are mostly active on the west coast (California, Oregon, and Washington), North central( Minnesota and Wisconsin) and the Northeast(New England states and Pennsylvania). It appears that ACHP members are focusing more plans of nmetropolitan areas. For example, in texas, it appears that only the wider Dallas area has plans while the more rural counties do not. When looking at the count of ACHP member plans that serve more than 15% fair or poor health, UPMC has the most at seven, kaiser at six, Security at four, and Health Alliance at four. the scatter plot shows a weak to no correlation between % fair or poor health and weighted stars. the correlation value is 0.0553. Most of the wieghted stars can be found between 12 and 18% fair or poor health. For the count of ACHP member plans that have greater than 15% uninsured, Indendent has three, Communitycare has two, and Avmed, Kaiser, and Aultcare have one. The scatter plot shows a weak correlation between % uninsured and weighted ACHP member stars. Most of the ACHP contracts can be found between five and 10 percent uninsured.For the bar chart of count of ACHP member plans that serve areas greater than 5%, UPMC has five, Healthparters, Ucare, and Kaiser have three and Security has two. The scatterplot has a slightly higher correlation than the other risk factors at 0.1415 and most of the ACHP contracts can be found between three and five percent unemployed. 

Overall, the ACHP member plan who is servicing the most areas that have high social risk factors is Kaiser followed by HealthAlliance and Independent. Kaiser also has the highest 2021 Overall Stars rating, most quality scores that are either 4s or 5s, and the most health contracts of any ACHP member. The member plans that are not focusing on high social risk areas are CDPHD, Martinspoint, PacificSource, Fallon, and HarvardPilgrim. However, CDPHP and Martin's point did have a high star rating for their 2021 Overall quality measure Since the weighted stars has such a low correlation to social risk factors and the choropleth map showed a focus of ACHP member plans in metropolitian areas, It may be of useful to include percent of urban or rural in the weighted stars in a future analysis.  

[Jake's Zenodo](https://zenodo.org/record/4323129)

[Jake's github](https://github.com/Jcolt2997/DATS-6103-Individual-Project-3-Jake-Lieberfarb)

[Jake's github.io](https://jcolt2997.github.io/DATS-6103-Individual-Project-3-Jake-Lieberfarb-githubio/)